# RAG Attack - Mode Démo (Sans Azure)

Ce notebook permet d'explorer les concepts d'agentic RAG sans avoir besoin des ressources Azure déployées.

## Installation des dépendances

In [ ]:
# Installation minimale pour le mode démo
!pip install langchain langchain-community langgraph -q

In [ ]:
import sys
from pathlib import Path

# Ajouter le package au path
sys.path.insert(0, str(Path.cwd()))

print("✅ Environnement configuré pour le mode démo")

## 1. Agent Simple avec Outils Mock

In [ ]:
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import operator

# Création d'outils mock
@tool
def recherche_produits(query: str) -> str:
    """Recherche des informations sur les produits VéloCorp"""
    catalogue = {
        "urban": {
            "nom": "VéloCity Urban",
            "prix": "1,500€",
            "autonomie": "50 km",
            "poids": "22 kg",
            "description": "Parfait pour les trajets quotidiens en ville"
        },
        "sport": {
            "nom": "VéloSport Pro",
            "prix": "2,500€",
            "autonomie": "80 km",
            "poids": "19 kg",
            "description": "Pour les sportifs exigeants"
        },
        "elite": {
            "nom": "VéloElite X",
            "prix": "4,500€",
            "autonomie": "120 km",
            "poids": "17 kg",
            "description": "Le summum de la technologie vélo électrique"
        }
    }
    
    results = []
    query_lower = query.lower()
    
    for key, product in catalogue.items():
        if key in query_lower or product["nom"].lower() in query_lower or "tous" in query_lower or "modèles" in query_lower:
            results.append(f"- {product['nom']}: {product['prix']}, autonomie {product['autonomie']}, {product['description']}")
    
    return "\n".join(results) if results else "Aucun produit trouvé"

@tool
def recherche_ventes(periode: str) -> str:
    """Recherche les données de ventes"""
    return f"""
    Ventes pour {periode}:
    - Urban: 127 unités (190,500€)
    - Sport: 89 unités (222,500€)
    - Elite: 34 unités (153,000€)
    Total: 250 unités (566,000€)
    Croissance: +15% vs période précédente
    """

print("✅ Outils mock créés")

In [ ]:
# Création d'un agent simple
class SimpleAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    
def create_simple_agent(tools):
    """Crée un agent simple avec des outils"""
    
    workflow = StateGraph(SimpleAgentState)
    
    def process_request(state):
        messages = state["messages"]
        user_input = messages[-1].content
        
        # Logique simple pour décider quel outil utiliser
        response_parts = []
        
        if "produit" in user_input.lower() or "vélo" in user_input.lower() or "modèle" in user_input.lower():
            result = recherche_produits.invoke(user_input)
            response_parts.append(f"📦 Produits trouvés:\n{result}")
        
        if "vente" in user_input.lower() or "chiffre" in user_input.lower():
            result = recherche_ventes.invoke("dernier trimestre")
            response_parts.append(f"📊 Données de ventes:\n{result}")
        
        if not response_parts:
            response_parts.append("Je peux vous aider avec les produits VéloCorp et les données de ventes. Que souhaitez-vous savoir?")
        
        return {"messages": [AIMessage(content="\n\n".join(response_parts))]}
    
    workflow.add_node("process", process_request)
    workflow.set_entry_point("process")
    workflow.add_edge("process", END)
    
    return workflow.compile()

# Test de l'agent
agent = create_simple_agent([recherche_produits, recherche_ventes])

# Question test
response = agent.invoke({"messages": [HumanMessage(content="Quels sont les modèles de vélos disponibles?")]})
print("Question: Quels sont les modèles de vélos disponibles?")
print("\nRéponse de l'agent:")
print(response["messages"][-1].content)

## 2. Agent ReAct (Reasoning and Acting)

In [ ]:
class ReActState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    thoughts: list
    observations: list

def create_react_agent(tools):
    """Crée un agent ReAct avec raisonnement explicite"""
    
    workflow = StateGraph(ReActState)
    
    def think(state):
        """Phase de réflexion"""
        messages = state["messages"]
        user_input = messages[-1].content if messages else ""
        
        # Simuler la réflexion
        thought = f"Je dois analyser la demande: '{user_input}'."
        
        if "produit" in user_input.lower() or "vélo" in user_input.lower():
            thought += " L'utilisateur cherche des informations sur les produits."
        if "vente" in user_input.lower():
            thought += " L'utilisateur veut des données de ventes."
        if "comparer" in user_input.lower():
            thought += " Je dois faire une comparaison."
            
        return {"thoughts": [thought]}
    
    def act(state):
        """Phase d'action"""
        thoughts = state.get("thoughts", [])
        messages = state["messages"]
        user_input = messages[-1].content if messages else ""
        
        observations = []
        
        # Agir en fonction de la réflexion
        if thoughts and "produits" in thoughts[-1]:
            result = recherche_produits.invoke(user_input)
            observations.append(f"J'ai trouvé les produits suivants: {result}")
        
        if thoughts and "ventes" in thoughts[-1]:
            result = recherche_ventes.invoke("dernier trimestre")
            observations.append(f"Voici les données de ventes: {result}")
            
        return {"observations": observations}
    
    def respond(state):
        """Phase de réponse"""
        thoughts = state.get("thoughts", [])
        observations = state.get("observations", [])
        
        response = "🧠 **Processus de raisonnement ReAct:**\n\n"
        
        for i, (thought, obs) in enumerate(zip(thoughts, observations), 1):
            response += f"**Étape {i}:**\n"
            response += f"💭 *Réflexion:* {thought}\n"
            response += f"👁️ *Observation:* {obs}\n\n"
        
        response += "**📝 Réponse finale:**\n"
        if observations:
            response += observations[-1]
        else:
            response += "Je n'ai pas trouvé d'information pertinente."
            
        return {"messages": [AIMessage(content=response)]}
    
    # Construction du graphe
    workflow.add_node("think", think)
    workflow.add_node("act", act)
    workflow.add_node("respond", respond)
    
    workflow.set_entry_point("think")
    workflow.add_edge("think", "act")
    workflow.add_edge("act", "respond")
    workflow.add_edge("respond", END)
    
    return workflow.compile()

# Test de l'agent ReAct
react_agent = create_react_agent([recherche_produits, recherche_ventes])

response = react_agent.invoke({
    "messages": [HumanMessage(content="Quels sont les produits et leurs ventes?")],
    "thoughts": [],
    "observations": []
})

print("Question: Quels sont les produits et leurs ventes?")
print("\n" + "="*60)
print(response["messages"][-1].content)

## 3. Planner Hiérarchique

In [ ]:
class PlannerState(TypedDict):
    objective: str
    plan: list
    current_step: int
    results: list
    final_output: str

def create_planner():
    """Crée un planner hiérarchique"""
    
    workflow = StateGraph(PlannerState)
    
    def create_plan(state):
        """Créer un plan d'exécution"""
        objective = state["objective"]
        
        # Créer un plan simple basé sur l'objectif
        plan = []
        
        if "rapport" in objective.lower() or "complet" in objective.lower():
            plan = [
                {"id": 1, "task": "Analyser les produits disponibles", "tool": "recherche_produits"},
                {"id": 2, "task": "Collecter les données de ventes", "tool": "recherche_ventes"},
                {"id": 3, "task": "Analyser la performance", "tool": "analysis"},
                {"id": 4, "task": "Générer des recommandations", "tool": "recommendations"}
            ]
        elif "produit" in objective.lower():
            plan = [{"id": 1, "task": "Rechercher les produits", "tool": "recherche_produits"}]
        elif "vente" in objective.lower():
            plan = [{"id": 1, "task": "Analyser les ventes", "tool": "recherche_ventes"}]
        else:
            plan = [{"id": 1, "task": "Traiter la demande générale", "tool": "general"}]
            
        return {"plan": plan, "current_step": 0}
    
    def execute_step(state):
        """Exécuter une étape du plan"""
        plan = state["plan"]
        current_step = state["current_step"]
        results = state.get("results", [])
        
        if current_step < len(plan):
            step = plan[current_step]
            
            # Simuler l'exécution
            if step["tool"] == "recherche_produits":
                result = recherche_produits.invoke("tous les modèles")
            elif step["tool"] == "recherche_ventes":
                result = recherche_ventes.invoke("dernier trimestre")
            elif step["tool"] == "analysis":
                result = "Performance: Croissance de 15%, modèle Elite le plus rentable"
            elif step["tool"] == "recommendations":
                result = "Recommandations: 1) Augmenter la production du modèle Elite, 2) Promouvoir le modèle Urban en ville"
            else:
                result = f"Exécution de: {step['task']}"
                
            results.append({"step": step["task"], "result": result})
            
        return {"results": results, "current_step": current_step + 1}
    
    def should_continue(state):
        """Vérifier si on doit continuer"""
        plan = state["plan"]
        current_step = state["current_step"]
        
        if current_step < len(plan):
            return "execute"
        else:
            return "finalize"
    
    def finalize(state):
        """Finaliser et créer le rapport"""
        results = state.get("results", [])
        objective = state["objective"]
        
        output = f"📋 **Rapport pour l'objectif:** {objective}\n\n"
        output += "**Plan exécuté:**\n"
        
        for i, r in enumerate(results, 1):
            output += f"\n✅ Étape {i}: {r['step']}\n"
            output += f"   Résultat: {r['result'][:100]}...\n"
        
        output += "\n**Synthèse:**\n"
        output += "Toutes les étapes ont été complétées avec succès."
        
        return {"final_output": output}
    
    # Construction du graphe
    workflow.add_node("plan", create_plan)
    workflow.add_node("execute", execute_step)
    workflow.add_node("finalize", finalize)
    
    workflow.set_entry_point("plan")
    workflow.add_edge("plan", "execute")
    workflow.add_conditional_edges(
        "execute",
        should_continue,
        {
            "execute": "execute",
            "finalize": "finalize"
        }
    )
    workflow.add_edge("finalize", END)
    
    return workflow.compile()

# Test du planner
planner = create_planner()

result = planner.invoke({
    "objective": "Créer un rapport complet sur les produits et les ventes",
    "plan": [],
    "current_step": 0,
    "results": [],
    "final_output": ""
})

print(result["final_output"])

## 4. Comparaison des Approches

In [ ]:
# Comparer les trois approches sur la même question
test_question = "Analyse les produits VéloCorp et leurs performances de vente"

print("📝 Question test:", test_question)
print("\n" + "="*60)

# 1. Agent Simple
print("\n1️⃣ AGENT SIMPLE:")
print("-"*40)
simple_response = agent.invoke({"messages": [HumanMessage(content=test_question)]})
print(simple_response["messages"][-1].content[:300] + "...")

# 2. Agent ReAct
print("\n2️⃣ AGENT REACT:")
print("-"*40)
react_response = react_agent.invoke({
    "messages": [HumanMessage(content=test_question)],
    "thoughts": [],
    "observations": []
})
print(react_response["messages"][-1].content[:300] + "...")

# 3. Planner
print("\n3️⃣ PLANNER HIÉRARCHIQUE:")
print("-"*40)
planner_response = planner.invoke({
    "objective": test_question,
    "plan": [],
    "current_step": 0,
    "results": [],
    "final_output": ""
})
print(planner_response["final_output"][:300] + "...")

print("\n" + "="*60)
print("\n💡 Observations:")
print("- L'agent simple est direct mais basique")
print("- ReAct montre son processus de raisonnement")
print("- Le planner décompose en étapes structurées")

## 5. Exercices Pratiques

In [ ]:
# Exercice 1: Créez votre propre outil
@tool
def votre_outil_personnalise(input: str) -> str:
    """Décrivez ce que fait votre outil"""
    # TODO: Implémentez votre logique ici
    return f"Résultat pour: {input}"

# Exercice 2: Créez un agent spécialisé
def create_specialized_agent():
    """TODO: Créez votre agent spécialisé"""
    pass

# Exercice 3: Améliorez le planner
# Ajoutez la gestion des dépendances entre tâches
# Ajoutez la gestion d'erreurs
# Ajoutez la possibilité de re-planifier

print("✏️ Exercices prêts à être complétés!")

## Conclusion

Ce notebook démo vous a montré les concepts clés de l'agentic RAG:

1. **Agents avec outils** - Comment les agents utilisent des outils pour accomplir des tâches
2. **ReAct** - L'importance de la transparence du raisonnement
3. **Planification** - La décomposition de tâches complexes
4. **Orchestration** - La coordination de multiples composants

### Prochaines étapes

Pour une expérience complète avec de vraies données:
1. Déployez les ressources Azure: `cd scai_onepoint_rag && make setup-all`
2. Utilisez le notebook principal: `rag_attack_partie2_agentic.ipynb`

### Ressources

- [Documentation LangGraph](https://python.langchain.com/docs/langgraph)
- [Documentation LangChain](https://python.langchain.com/)
- [Papers ReAct](https://arxiv.org/abs/2210.03629)